# Notebook 02: Data Cleaning

This notebook focuses on preparing the raw datasets for analysis by addressing missing values, correcting data inconsistencies, and creating new features. The goal is to ensure the data is clean, standardized, and ready for modeling while retaining relevant information for fairness and bias analysis.

In [1]:
import pandas as pd
import numpy as np

from datetime import datetime

import warnings
warnings.filterwarnings('ignore')  # Ignore all warnings

In [2]:
# Display all rows and columns
pd.set_option('display.max_colwidth', None)  # Show full content in each cell
pd.set_option('display.max_rows', None)      # Show all rows
pd.set_option('display.max_columns', None)   # Show all columns

In [3]:
# Load datasets
tbl_action_df = pd.read_csv("../data/raw/tbl_action.csv")
tbl_employee_df = pd.read_csv("../data/raw/tbl_employee.csv")
tbl_perf_df = pd.read_csv("../data/raw/tbl_perf.csv")
hr_data_df = pd.read_csv("../data/raw/hr_data.csv")

In [4]:
# Verify data loading
print("tbl_action_df:")
display(tbl_action_df.head())

print("\ntbl_employee_df:")
display(tbl_employee_df.head())

print("\ntbl_perf_df:")
display(tbl_perf_df.head())

print("\nhr_data_df:")
display(hr_data_df.head())

tbl_action_df:


,ActID,ActionID,EmpID,EffectiveDt
0,1,10,1,01/Jan/2015
1,2,10,2,01/Jan/2015
2,3,10,3,06/Oct/2015
3,4,10,4,01/Jan/2015
4,5,10,5,22/May/2015



tbl_employee_df:


,EmpID,EmpName,EngDt,TermDt,DepID,GenderID,RaceID,MgrID,DOB,PayRate,Level
0,1,"Nash, Kay K",01/01/2015,29/May/2017,8,1,1,1,12/Feb/1987,NaN,1
1,2,"Simpson, Raymond X",01/01/2015,NaN,7,0,4,2,15/Nov/1982,NaN,1
2,3,"Devlin, Kay M",06/10/2015,24/Jun/2019,10,1,5,3,22/Feb/1986,NaN,1
3,4,"Massie, Elisabeth V",01/01/2015,NaN,4,1,2,4,25/May/1977,NaN,1
4,5,"Mead, Catherine N",22/05/2015,15/Jul/2020,8,1,1,1,04/Apr/1995,NaN,2



tbl_perf_df:


,PerfID,EmpID,Rating,PerfDate
0,1,1,1,31/Dec/2015
1,2,2,2,31/Dec/2015
2,3,3,2,31/Dec/2015
3,4,4,4,31/Dec/2015
4,5,5,2,31/Dec/2015



hr_data_df:


,Employee_Name,EmpID,MarriedID,MaritalStatusID,GenderID,EmpStatusID,DeptID,PerfScoreID,FromDiversityJobFairID,PayRate,Termd,PositionID,Position,State,Zip,DOB,Sex,MaritalDesc,CitizenDesc,HispanicLatino,RaceDesc,DateofHire,DateofTermination,TermReason,EmploymentStatus,Department,ManagerName,ManagerID,RecruitmentSource,PerformanceScore,EngagementSurvey,EmpSatisfaction,SpecialProjectsCount,LastPerformanceReview_Date,DaysLateLast30,Original DS
0,"Gonzalez, Maria",1988299991,0,3,0,1,3,3,0,39.0,0,14,IT Support,MA,2472.0,16-04-81,F,Separated,US Citizen,Yes,White,05-01-15,NaN,N/A - still employed,Active,IT/IS,Peter Monroe,7.0,Employee Referral,Fully Meets,4.61,4,5,28-01-19,0.0,Orig
1,"Cockel, James",1599991009,0,0,1,1,5,3,0,15.0,0,19,Production Technician I,MA,2452.0,08-09-77,M,Single,US Citizen,No,White,08-07-13,NaN,N/A - still employed,Active,Production,David Stanley,14.0,Employee Referral,Fully Meets,1.51,5,0,14-01-19,0.0,Orig
2,"Bunbury, Jessica",1504073368,1,1,0,5,6,3,0,55.0,1,3,Area Sales Manager,VA,21851.0,01-06-64,F,Married,Eligible NonCitizen,No,Black or African American,15-08-11,02-08-14,Another position,Voluntarily Terminated,Sales,John Smith,17.0,Social Networks - Facebook Twitter etc,Fully Meets,3.14,5,0,NaN,NaN,Orig
3,"Buck, Edward",1504073313,1,1,1,1,6,3,0,55.0,0,3,Area Sales Manager,MA,2330.0,07-07-75,M,Married,US Citizen,No,White,29-09-14,NaN,N/A - still employed,Active,Sales,Lynn Daneault,21.0,Employee Referral,Fully Meets,3.31,5,0,28-01-19,0.0,Orig
4,"Jacobi, Hannah",1503072857,0,2,0,1,5,3,0,21.0,0,19,Production Technician I,MA,1778.0,22-03-66,F,Divorced,US Citizen,No,White,30-09-13,NaN,N/A - still employed,Active,Production,Kissy Sullivan,20.0,Employee Referral,Fully Meets,2.05,5,0,22-02-19,0.0,Orig


In [5]:
def replace_null(df):
    df.replace(['NaN', 'null', 'None', '', ' '], pd.NA, inplace=True)
    return df

In [6]:
def handle_missing_values(df):
    if 'TermReason' in df.columns:
        df['TermReason'] = df['TermReason'].fillna('Unknown')
    if 'ManagerName' in df.columns:
        df['ManagerName'] = df['ManagerName'].fillna('Unknown')
    if 'DateofTermination' in df.columns:
        df['DateofTermination'] = df['DateofTermination'].fillna('Still Employed')
    return df

In [7]:
def transform_dates(df, date_columns):
    for col in date_columns:
        if col in df.columns:
            df[col] = pd.to_datetime(df[col], errors='coerce', dayfirst=True)
            if col == 'DOB':
                current_year = datetime.now().year
                df.loc[df[col].dt.year > current_year, col] -= pd.offsets.DateOffset(years=100)
    return df

In [8]:
def handle_duplicates(df, subset_columns):
    if subset_columns:
        df.drop_duplicates(subset=subset_columns, inplace=True)
    return df

In [9]:
def convert_data_types(df, category_columns, numeric_columns):
    for col in category_columns:
        if col in df.columns:
            df[col] = df[col].astype('category')
    for col in numeric_columns:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')
    return df

In [10]:
def create_new_features(df):
    if 'DateofTermination' in df.columns and 'DateofHire' in df.columns:
        df['Churn'] = ~df['DateofTermination'].isna()
        df['Tenure'] = (df['DateofTermination'].fillna(pd.Timestamp.now()) - df['DateofHire']).dt.days
    if 'DOB' in df.columns:
        df['Age'] = (pd.Timestamp.now() - df['DOB']).dt.days // 365
    if 'Churn' in df.columns:
        df['Churn-Yes/No'] = df['Churn'].apply(lambda x: 'Yes' if x else 'No')
    return df

In [11]:
def feature_engineering(df):
    if 'PerformanceScoreNumeric' in df.columns:
        df['GoodFit'] = (
            (df['PerformanceScoreNumeric'] >= 3) &
            ~((df['Churn'] == 1) & (df['Tenure'] < 365))
        ).astype(int)
    return df

In [12]:
def clean_categorical_data(df):
    if 'Sex' in df.columns:
        df['Sex'] = df['Sex'].str.strip()
    for col in df.select_dtypes(include='category').columns:
        df[col] = df[col].cat.remove_unused_categories()
    return df

In [13]:
def drop_high_null_columns(df, columns_to_drop):
    df.drop(columns=columns_to_drop, errors='ignore', inplace=True)
    return df

In [14]:
def drop_unnecessary_columns(df, columns_to_drop):
    df.drop(columns=columns_to_drop, errors='ignore', inplace=True)
    return df

In [15]:
def filter_rows(df, conditions):
    for condition in conditions:
        df = df.query(condition)
    return df

In [16]:
def class_balance_analysis(df, target_column):
    if target_column in df.columns:
        print(df[target_column].value_counts(normalize=True))

In [17]:
def calculate_avg_ratings(tbl_perf_df):
    return tbl_perf_df.groupby('EmpID')['Rating'].median().reset_index(name='AvgRating')

In [18]:
# Execution

In [19]:
# Clean tbl_action_df
tbl_action_df = replace_null(tbl_action_df)
tbl_action_df = transform_dates(tbl_action_df, date_columns=['EffectiveDt'])
tbl_action_df = handle_duplicates(tbl_action_df, subset_columns=['ActID'])
tbl_action_df = convert_data_types(tbl_action_df, category_columns=[], numeric_columns=['ActionID', 'EmpID'])

In [20]:
tbl_action_df.head()

,ActID,ActionID,EmpID,EffectiveDt
0,1,10,1,2015-01-01
1,2,10,2,2015-01-01
2,3,10,3,2015-10-06
3,4,10,4,2015-01-01
4,5,10,5,2015-05-22


In [21]:
# Clean tbl_employee_df
tbl_employee_df = replace_null(tbl_employee_df)
tbl_employee_df = handle_missing_values(tbl_employee_df)
tbl_employee_df = transform_dates(tbl_employee_df, date_columns=['DOB', 'EngDt', 'TermDt'])
tbl_employee_df = handle_duplicates(tbl_employee_df, subset_columns=['EmpID'])
tbl_employee_df = convert_data_types(tbl_employee_df, category_columns=['GenderID', 'RaceID'], numeric_columns=['PayRate', 'Level'])
tbl_employee_df = create_new_features(tbl_employee_df)
tbl_employee_df = feature_engineering(tbl_employee_df)
tbl_employee_df = clean_categorical_data(tbl_employee_df)
tbl_employee_df = drop_high_null_columns(tbl_employee_df, columns_to_drop=['Zip', 'LastPerformanceReview_Date', 'DaysLateLast30'])
tbl_employee_df = drop_unnecessary_columns(tbl_employee_df, columns_to_drop=['Employee_Name', 'State', 'ManagerName', 'ManagerID'])

In [22]:
tbl_employee_df.head()

,EmpID,EmpName,EngDt,TermDt,DepID,GenderID,RaceID,MgrID,DOB,PayRate,Level,Age
0,1,"Nash, Kay K",2015-01-01,2017-05-29,8,1,1,1,1987-02-12,NaN,1,37
1,2,"Simpson, Raymond X",2015-01-01,NaT,7,0,4,2,1982-11-15,NaN,1,42
2,3,"Devlin, Kay M",2015-10-06,NaT,10,1,5,3,1986-02-22,NaN,1,38
3,4,"Massie, Elisabeth V",2015-01-01,NaT,4,1,2,4,1977-05-25,NaN,1,47
4,5,"Mead, Catherine N",2015-05-22,NaT,8,1,1,1,1995-04-04,NaN,2,29


In [23]:
# Clean tbl_perf_df
tbl_perf_df = replace_null(tbl_perf_df)
tbl_perf_df = transform_dates(tbl_perf_df, date_columns=['PerfDate'])
tbl_perf_df = handle_duplicates(tbl_perf_df, subset_columns=['PerfID'])
tbl_perf_df = convert_data_types(tbl_perf_df, category_columns=[], numeric_columns=['EmpID', 'Rating'])

In [24]:
tbl_perf_df.head()

,PerfID,EmpID,Rating,PerfDate
0,1,1,1,2015-12-31
1,2,2,2,2015-12-31
2,3,3,2,2015-12-31
3,4,4,4,2015-12-31
4,5,5,2,2015-12-31


In [25]:
# Clean hr_data_df
hr_data_df = replace_null(hr_data_df)
hr_data_df = handle_missing_values(hr_data_df)
hr_data_df = transform_dates(hr_data_df, date_columns=['DOB', 'DateofHire', 'DateofTermination', 'LastPerformanceReview_Date'])
hr_data_df = handle_duplicates(hr_data_df, subset_columns=['EmpID'])
hr_data_df = convert_data_types(
    hr_data_df,
    category_columns=['MaritalStatusID', 'GenderID', 'EmpStatusID', 'DeptID', 'PerfScoreID'],
    numeric_columns=['PayRate', 'EngagementSurvey', 'EmpSatisfaction']
)

# Map PerformanceScore to numeric values
performance_score_mapping = {
    'Exceeds': 4,
    'Fully Meets': 3,
    'Needs Improvement': 2,
    'PIP': 1
}
hr_data_df['PerformanceScoreNumeric'] = hr_data_df['PerformanceScore'].map(performance_score_mapping)

hr_data_df = create_new_features(hr_data_df)
hr_data_df = feature_engineering(hr_data_df)
hr_data_df = clean_categorical_data(hr_data_df)
hr_data_df = drop_high_null_columns(hr_data_df, columns_to_drop=['DaysLateLast30', 'Original DS'])
hr_data_df = drop_unnecessary_columns(hr_data_df, columns_to_drop=['ManagerName', 'RecruitmentSource'])

# Filter rows (e.g., exclude 'Future Start')
if 'EmploymentStatus' in hr_data_df.columns:
    hr_data_df = filter_rows(hr_data_df, conditions=["EmploymentStatus != 'Future Start'"])

In [26]:
# Perform class balance analysis
class_balance_analysis(hr_data_df, target_column='GoodFit')

GoodFit
1    0.81853
0    0.18147
Name: proportion, dtype: float64


In [27]:
columns_to_drop = [
    'EmpID', 'ManagerID', 'ManagerName', 'RecruitmentSource', 'PayRate',
    'MaritalDesc', 'EngagementSurvey', 'EmpSatisfaction', 'MarriedID',
    'SpecialProjectsCount', 'TermReason', 'EmploymentStatus', 'Churn',
    'LastPerformanceReview_Date', 'DaysLateLast30', 'Original DS',
    'MaritalStatusID', 'EmpStatusID', 'DeptID', 'PerfScoreID', 'PerformanceScore',
    'FromDiversityJobFairID', 'GenderID', 'Termd', 'PositionID', 'Tenure',
    'Churn-Yes/No'
]
hr_data_df = hr_data_df.drop(columns=columns_to_drop, errors='ignore')

In [28]:
# Calculate Years of Experience
hr_data_df['YearsExperience'] = (
    (pd.Timestamp.now() - hr_data_df['DateofHire']).dt.days / 365
).fillna(0).astype(int)
hr_data_df = hr_data_df.drop(columns=['DateofHire', 'DateofTermination'])

In [29]:
# Bucket Age into ranges
age_bins = [0, 30, 50, 100]
age_labels = ['<30', '30-50', '>50']
hr_data_df['AgeGroup'] = pd.cut(hr_data_df['Age'], bins=age_bins, labels=age_labels)
hr_data_df = hr_data_df.drop(columns=['DOB', 'Age'])

In [30]:
hr_data_df.Position.value_counts()

Position
Production Technician I         1422
Production Technician II         523
Area Sales Manager               259
Production Manager               154
IT Support                        88
Software Engineer                 88
Data Analyst                      77
Network Engineer                  55
Database Administrator            55
Sr. Network Engineer              48
BI Developer                      44
Administrative Assistant          33
Accountant I                      33
Sales Manager                     30
Senior BI Developer               30
IT Manager - DB                   22
Shared Services Manager           20
IT Director                       11
CIO                               11
Data Analyst                      11
IT Manager - Support              11
IT Manager - Infra                11
Data Architect                    11
BI Director                       11
Director of Sales                 11
Director of Operations            11
President & CEO              

In [31]:
# Define bins and labels for experience categories
experience_bins = [0, 2, 5, 10, 20, float('inf')]  # Define the ranges for bins
experience_labels = ['0-2 years', '3-5 years', '6-10 years', '11-20 years', '>20 years']  # Corresponding labels

# Apply bins to create a new ExperienceCategory column
hr_data_df['ExperienceCategory'] = pd.cut(
    hr_data_df['YearsExperience'], 
    bins=experience_bins, 
    labels=experience_labels, 
    right=False  # Include left edge in the interval
)

In [32]:
display(hr_data_df.head())

,Employee_Name,Position,State,Zip,Sex,CitizenDesc,HispanicLatino,RaceDesc,Department,PerformanceScoreNumeric,GoodFit,YearsExperience,AgeGroup,ExperienceCategory
0,"Gonzalez, Maria",IT Support,MA,2472.0,F,US Citizen,Yes,White,IT/IS,3,1,10,30-50,11-20 years
1,"Cockel, James",Production Technician I,MA,2452.0,M,US Citizen,No,White,Production,3,1,11,30-50,11-20 years
2,"Bunbury, Jessica",Area Sales Manager,VA,21851.0,F,Eligible NonCitizen,No,Black or African American,Sales,3,1,13,>50,11-20 years
3,"Buck, Edward",Area Sales Manager,MA,2330.0,M,US Citizen,No,White,Sales,3,1,10,30-50,11-20 years
4,"Jacobi, Hannah",Production Technician I,MA,1778.0,F,US Citizen,No,White,Production,3,1,11,>50,11-20 years


In [33]:
hr_data_df.to_parquet("../data/interim/hr_data.parquet")